In [1]:
import os
os.environ['SPARK_NAME'] = "/opt/spark"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
# os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'

# można też spróbować wykorzystać moduł findspark do automatycznego odnalezienia miejsca instalacji sparka
# import findspark
# findspark.init()
# lub
# findspark.init("/opt/spark")

In [2]:
from pyspark.sql import SparkSession

# spark = SparkSession.builder.master("spark://spark-master:7077").appName("Create-DataFrame").getOrCreate()
# konfiguracja z określeniem liczby wątków (2) oraz ilości pamięci do wykorzystania poza stertą interpretera Pythona
spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Create-DataFrame")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","6g")\
        .getOrCreate()
spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/16 16:41:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[2] appName=Create-DataFrame>

In [3]:
sc = spark.sparkContext

In [4]:
# pobranie spakowanego zbioru za pomocą polecenia systemowego wget
# strona datasetu: https://archive.ics.uci.edu/dataset/911/recipe+reviews+and+user+feedback+dataset
!wget https://archive.ics.uci.edu/static/public/911/recipe+reviews+and+user+feedback+dataset.zip

--2024-12-16 16:41:35--  https://archive.ics.uci.edu/static/public/911/recipe+reviews+and+user+feedback+dataset.zip
128.195.10.252ive.ics.uci.edu (archive.ics.uci.edu)... 
connected. to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... 
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘recipe+reviews+and+user+feedback+dataset.zip’

recipe+reviews+and+     [      <=>           ]   2.02M  1.41MB/s    in 1.4s    

2024-12-16 16:41:37 (1.41 MB/s) - ‘recipe+reviews+and+user+feedback+dataset.zip’ saved [2114088]



In [5]:
# listujemy zawartość bieżącego folderu
!ls

data	      Lab6.ipynb				    Zad1.ipynb
employee.csv  recipe+reviews+and+user+feedback+dataset.zip  Zad2.ipynb
lab	      recipe_reviews.zip


In [6]:
# zmiana nazwy pliku - nie jest konieczna, ale trzeba zmienić później ścieżkę w kolejnej komórce notatnika
!mv recipe+reviews+and+user+feedback+dataset.zip recipe_reviews.zip

In [7]:
# wypakowujemy plik do podfolderu data
import zipfile
with zipfile.ZipFile("recipe_reviews.zip", 'r') as zip_ref:
    zip_ref.extractall("./data")

In [8]:
!ls ./data

 parquet  'Recipe Reviews and User Feedback Dataset.csv'


In [9]:
# sprawdzamy jak wyglądają 3 pierwsze linie pliku, widać, że pierwsza zawiera nagłówki kolumn a dane są oddzielone przecinkiem
!head -3 "data/Recipe Reviews and User Feedback Dataset.csv"

,recipe_number,recipe_code,recipe_name,comment_id,user_id,user_name,user_reputation,created_at,reply_count,thumbs_up,thumbs_down,stars,best_score,text
0,001,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2G3aneMRgRMZwXqIHmSdXSG1hEM,u_9iFLIhMa8QaG,Jeri326,1,1665619889,0,0,0,5,527,"I tweaked it a little, removed onions because of onion haters in my house, used Italian seasoning instead of just oregano, and use a paprika/ cayenne mix and a little more than the recipe called for.. we like everything a bit more hot. The chili was amazing! It was easy to make and everyone absolutely loved it. It will now be a staple meal in our house."
1,001,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2FsPC83HtzCsQAtOxlbL6RcaPbY,u_Lu6p25tmE77j,Mark467,50,1665277687,0,7,0,5,724,"Bush used to have a white chili bean and it made this recipe super simple. I’ve written to them and asked them to please!, bring them back"


In [10]:
df_reviews = spark.read.csv('./data/Recipe Reviews and User Feedback Dataset.csv', header=True, sep=",")

In [11]:
# najpopularniejsza metoda ich pobrania to show(), ale jest ich więcej
df_reviews.show(5)

+---+-------------+-----------+------------------+--------------------+--------------+----------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
|_c0|recipe_number|recipe_code|       recipe_name|          comment_id|       user_id| user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|                text|
+---+-------------+-----------+------------------+--------------------+--------------+----------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
|  0|          001|      14299|Creamy White Chili|sp_aUSaElGf_14299...|u_9iFLIhMa8QaG|   Jeri326|              1|1665619889|          0|        0|          0|    5|       527|I tweaked it a li...|
|  1|          001|      14299|Creamy White Chili|sp_aUSaElGf_14299...|u_Lu6p25tmE77j|   Mark467|             50|1665277687|          0|        7|          0|    5|       724|Bush used to have...|
|  2|          

24/12/16 16:41:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/Lab6/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


In [12]:
# rzut oka na schemę tego DataFrame
df_reviews.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- recipe_number: string (nullable = true)
 |-- recipe_code: string (nullable = true)
 |-- recipe_name: string (nullable = true)
 |-- comment_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_reputation: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- reply_count: string (nullable = true)
 |-- thumbs_up: string (nullable = true)
 |-- thumbs_down: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- best_score: string (nullable = true)
 |-- text: string (nullable = true)



In [13]:
# widać, że wszystkie kolumny są typu string, to jest domyślny sposób wczytywania danych przez spark z plain text
# możemy jednak przekazać dodatkowy parametr, który na podstawie próbki danych spróbuje dobrać typ danych odpowiedni dla kolumny
df_reviews = spark.read.csv('./data/Recipe Reviews and User Feedback Dataset.csv', header=True, sep=",", inferSchema=True)

In [14]:
# po wypisaniu schemy widać zmianę
df_reviews.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- recipe_number: string (nullable = true)
 |-- recipe_code: string (nullable = true)
 |-- recipe_name: string (nullable = true)
 |-- comment_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_reputation: string (nullable = true)
 |-- created_at: integer (nullable = true)
 |-- reply_count: integer (nullable = true)
 |-- thumbs_up: integer (nullable = true)
 |-- thumbs_down: integer (nullable = true)
 |-- stars: integer (nullable = true)
 |-- best_score: integer (nullable = true)
 |-- text: string (nullable = true)



In [15]:
# ramkę możemy również inicjalizować wskazując pożądane typy danych
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType, LongType

data = [("James","","Smith",36636,"M",3000),
    ("Michael","Rose","",40288,"M",4000),
    ("Robert","","Williams",42114,"M",4000),
    ("Maria","Anne","Jones",39192,"F",4000),
    ("Jen","Mary","Brown","","F",1000)
  ]

schema = StructType([ \
    StructField("firstname", StringType(), True), \
    StructField("user_id", StringType(), True), \
    StructField("lastname", StringType(), True), \
    StructField("id", StringType(), True), \
    # błąd konwersji "" na int!
    # StructField("id", LongType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", StringType(), True)
    # chcielibyśmy tak, ale tutaj nie da się za bardzo - błąd konwersji int na decimal!
    # StructField("salary", DecimalType(10,2), True) \
  ])

df_test = spark.createDataFrame(data=data,schema=schema)
df_test.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)



In [16]:
# możemy wykonać rzutowanie po wczytaniu danych z większością kolumn typu tekstowego
import pyspark.sql.functions as F

df_test = df_test.withColumn("salary", F.col("salary").cast("decimal(10,2)"))
df_test.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: decimal(10,2) (nullable = true)



In [17]:
# wyświetlenie danych z pojedynczej kolumny
df_test.select(df_test.salary).show()

+-------+
| salary|
+-------+
|3000.00|
|4000.00|
|4000.00|
|4000.00|
|1000.00|
+-------+



In [18]:
# ile wierszy w ramce?
df_reviews.count()

18268

In [19]:
# DataFrame składa się z obiektów typu Column dla każdej kolumny
# API dla typu Column: https://spark.apache.org/docs/3.5.3/api/python/reference/pyspark.sql/api/pyspark.sql.Column.html

# do kolumn możemy się odwoływać tak jak w pandas API, ale wynik jest inny
df_reviews.user_name, df_reviews['user_name']

(Column<'user_name'>, Column<'user_name'>)

In [20]:
# aby wyświetlić dane musimy wywoałać funkcję select na obiekcie dataframe

df_reviews.select(df_reviews.user_name).show(5)

+----------+
| user_name|
+----------+
|   Jeri326|
|   Mark467|
|Barbara566|
|jeansch123|
|  camper77|
+----------+
only showing top 5 rows



In [21]:
# do funkcji select możemy przekazać wiele kolumn a wywołania podobnie jak dla RDD są leniwe
print(df_reviews.select(df_reviews.user_name, df_reviews.user_reputation))
# musimy więc wywołać funkcję, której wykonanie "zmusi" Sparka do wyliczenia jej wartości lub jawnie wywołać np. show
df_reviews.select(df_reviews.user_name, df_reviews.user_reputation).show(5)

DataFrame[user_name: string, user_reputation: string]
+----------+---------------+
| user_name|user_reputation|
+----------+---------------+
|   Jeri326|              1|
|   Mark467|             50|
|Barbara566|             10|
|jeansch123|              1|
|  camper77|             10|
+----------+---------------+
only showing top 5 rows



In [22]:
# można zmienić to domyślne zachowanie Spark, ale zazwyczaj nie jest to dobry pomysł, chyba, że zbiór jest mały
# zmieniamy to poprzez edycję poniższego parametru
# spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [23]:
# lub indeksując kolumny innym sposobem
df_reviews.select(df_reviews['user_name'],df_reviews['user_reputation']).show(5)

+----------+---------------+
| user_name|user_reputation|
+----------+---------------+
|   Jeri326|              1|
|   Mark467|             50|
|Barbara566|             10|
|jeansch123|              1|
|  camper77|             10|
+----------+---------------+
only showing top 5 rows



In [24]:
from pyspark.sql.functions import isnan, when, count, col

# policzymy teraz liczbę wartości NULL w każdej kolumnie
df_reviews.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_reviews.columns]).show()

24/12/16 16:41:48 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/Lab6/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv
[Stage 12:=============================>                            (1 + 1) / 2]

+---+-------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+
|_c0|recipe_number|recipe_code|recipe_name|comment_id|user_id|user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|text|
+---+-------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+
|  0|           45|         64|         74|        77|     80|       83|             84|        86|         86|       86|         86|   86|        86|  86|
+---+-------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+



In [25]:
# rzućmy okiem na kilka wierszy gdzie w kolumnie recipe_name jest wartość NULL
df_reviews.filter(df_reviews.recipe_code.isNull()).show()

+--------------------+--------------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+
|                 _c0|       recipe_number|recipe_code|recipe_name|comment_id|user_id|user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|text|
+--------------------+--------------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+
|      Thank you!!!!"|                NULL|       NULL|       NULL|      NULL|   NULL|     NULL|           NULL|      NULL|       NULL|     NULL|       NULL| NULL|      NULL|NULL|
| It was excellent!  |                NULL|       NULL|       NULL|      NULL|   NULL|     NULL|           NULL|      NULL|       NULL|     NULL|       NULL| NULL|      NULL|NULL|
|The recipe was a ...|                NULL|       NULL|       NULL|      NULL|   NULL|     NULL|    

24/12/16 16:41:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/Lab6/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


In [26]:
# zapisanie do nowej ramki danych bez wartości pustych
df_reviews_clean = df_reviews.na.drop()
df_reviews_clean.count()

24/12/16 16:41:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/Lab6/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


18182

In [27]:
# dla pewności możemy to sprawdzić raz jeszcze
df_reviews_clean.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_reviews_clean.columns]).show()

24/12/16 16:41:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/Lab6/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


+---+-------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+
|_c0|recipe_number|recipe_code|recipe_name|comment_id|user_id|user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|text|
+---+-------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+
|  0|            0|          0|          0|         0|      0|        0|              0|         0|          0|        0|          0|    0|         0|   0|
+---+-------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+



In [28]:
# filtrowanie danych z ramki
df_reviews_clean.filter(df_reviews.user_name.startswith('a')).select(df_reviews_clean.user_name).show(10)
df_reviews_clean.filter(df_reviews.stars == 5).show(10)

+--------------+
|     user_name|
+--------------+
|         ahmom|
|  annamossburg|
|   astarzynski|
|     adamscook|
|      angela32|
|       annaf27|
|    avanhaasen|
|    aunt ann's|
|     angelic0w|
|a_n_g_e_l_0715|
+--------------+
only showing top 10 rows



24/12/16 16:41:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/Lab6/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv
24/12/16 16:41:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Ex

+---+-------------+-----------+------------------+--------------------+--------------+----------------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
|_c0|recipe_number|recipe_code|       recipe_name|          comment_id|       user_id|       user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|                text|
+---+-------------+-----------+------------------+--------------------+--------------+----------------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
|  0|          001|      14299|Creamy White Chili|sp_aUSaElGf_14299...|u_9iFLIhMa8QaG|         Jeri326|              1|1665619889|          0|        0|          0|    5|       527|I tweaked it a li...|
|  1|          001|      14299|Creamy White Chili|sp_aUSaElGf_14299...|u_Lu6p25tmE77j|         Mark467|             50|1665277687|          0|        7|          0|    5|       724|Bush us

In [29]:
from pyspark.sql.functions import avg

# wyliczenie średniej wartości z kolumny
df_reviews_clean.select(avg(df_reviews_clean.thumbs_up)).show()

24/12/16 16:41:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/Lab6/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


+------------------+
|    avg(thumbs_up)|
+------------------+
|1.0892641073589264|
+------------------+



In [30]:
# ale możemy się dowiedzieć tego i więcej w sposób podobny do tego z biblioteki pandas
df_reviews_clean.select(df_reviews_clean.thumbs_up).describe().show()

24/12/16 16:41:52 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/Lab6/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


+-------+------------------+
|summary|         thumbs_up|
+-------+------------------+
|  count|             18182|
|   mean|1.0892641073589264|
| stddev| 4.201003572820717|
|    min|                 0|
|    max|               106|
+-------+------------------+



In [31]:
from pyspark.sql.functions import desc

df_reviews_clean.groupby('recipe_code').agg({'thumbs_down': 'sum'}).sort(desc('sum(thumbs_down)')).show(10)

24/12/16 16:41:52 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/Lab6/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


+-----------+----------------+
|recipe_code|sum(thumbs_down)|
+-----------+----------------+
|       2832|             488|
|       9739|             354|
|      17826|             328|
|      18345|             313|
|      12003|             306|
|       4383|             301|
|      41095|             275|
|       8202|             272|
|       6504|             264|
|       6086|             262|
+-----------+----------------+
only showing top 10 rows



In [32]:
# deklaracja zbiorów wartości dla poszczególnych kolumn przyszłego zbioru danych
header = ['id', 'firstname', 'lastname', 'age', 'salary']
firstnames = ['Adam', 'Katarzyna', 'Krzysztof', 'Marek', 'Aleksandra', 'Zbigniew', 'Wojciech', 'Mieczysław', 'Agata', 'Wisława']
lastnames = ['Mieczykowski', 'Kowalski', 'Malinowski' , 'Szczaw', 'Glut', 'Barański', 'Brzęczyszczykiewicz', 'Wróblewski', 'Wlotka', 'Pysla']
age = {'min': 18, 'max': 68}
salary = {'min': 3200, 'max': 12500}

In [33]:
# funkcja do generowania fikcyjnego datasetu
# n_rows oznacza ilość wierszy, którą chcemy finalnie uzyskać


import random
from tqdm import tqdm

def build_dataset(filename, n_rows=100, chunk_size=100000):
    rows = []
    rows.append(header)
    mu = (salary['max'] + salary['min']) / 2
    sigma = 1000

    with open(filename, 'w', encoding='utf-8') as filehandler:
        
        for id in tqdm(range(1, n_rows + 1), total=n_rows, desc="Building dataset..."):
            row = [
                f'{id}', 
                f'{random.choice(firstnames)}', 
                f'{random.choice(lastnames)}', 
                f"{random.randint(age['min'], age['max'])}",
                f"{round(float(random.normalvariate(mu=mu, sigma=sigma)), 2)}"
            ]
            rows.append(row)
            if id % chunk_size == 0:
                filehandler.writelines([f"{','.join(row)}\n" for row in rows])
                rows = []

In [34]:
# około 715MB zostanie zapisanych w pliku csv, dostosuj ilość rekordów do swoich potrzeb
build_dataset('employee.csv', 20_000_000)

Building dataset...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20000000/20000000 [01:47<00:00, 185420.43it/s]


In [35]:
%%time
# więcej magicznych metod w Jupyter Notebooku: https://ipython.readthedocs.io/en/stable/interactive/magics.html
# wczytanie pliku csv przez spark
# df = spark.read.csv('employee.csv', header=True)
df = spark.read.csv('employee.csv', header=True, inferSchema=True)

[Stage 34:================================================>         (5 + 1) / 6]

CPU times: user 33.7 ms, sys: 9.75 ms, total: 43.5 ms
Wall time: 18.5 s


In [36]:
type(df)

pyspark.sql.dataframe.DataFrame

In [37]:
%%time
# wypisujemy schemat i 10 pierwszych wierszy utworzonego obiektu Spark DataFrame
df.printSchema()
df.show(10)

root
 |-- id: integer (nullable = true)
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: double (nullable = true)

+---+----------+-------------------+---+--------+
| id| firstname|           lastname|age|  salary|
+---+----------+-------------------+---+--------+
|  1|   Wisława|         Wróblewski| 67| 8835.52|
|  2|Aleksandra|              Pysla| 61| 7993.65|
|  3| Krzysztof|           Barański| 50| 8391.89|
|  4|      Adam|              Pysla| 48| 8744.56|
|  5| Katarzyna|         Malinowski| 44|  8324.4|
|  6|      Adam|           Kowalski| 27|  8850.1|
|  7|   Wisława|             Wlotka| 29| 8950.67|
|  8|Mieczysław|Brzęczyszczykiewicz| 32| 6973.38|
|  9| Krzysztof|Brzęczyszczykiewicz| 37|10180.01|
| 10| Katarzyna|             Wlotka| 26| 8365.36|
+---+----------+-------------------+---+--------+
only showing top 10 rows

CPU times: user 3.75 ms, sys: 0 ns, total: 3.75 ms
Wall time: 110 ms


In [38]:
# przykład wykorzystania funkcji transform, która mapuje wykonanie stworzonej funkcji tu_upper_str_columns na istniejącą kolumnę
# i zwraca nową ramkę z dodatkową kolumną
from pyspark.sql.functions import upper

def to_upper_str_columns(df, column_name, new_column_name):
    return df.withColumn(new_column_name, upper(df[column_name]))

df = df.transform(to_upper_str_columns, "firstname", "firstname_upper")

In [39]:
df.show(10)

+---+----------+-------------------+---+--------+---------------+
| id| firstname|           lastname|age|  salary|firstname_upper|
+---+----------+-------------------+---+--------+---------------+
|  1|   Wisława|         Wróblewski| 67| 8835.52|        WISŁAWA|
|  2|Aleksandra|              Pysla| 61| 7993.65|     ALEKSANDRA|
|  3| Krzysztof|           Barański| 50| 8391.89|      KRZYSZTOF|
|  4|      Adam|              Pysla| 48| 8744.56|           ADAM|
|  5| Katarzyna|         Malinowski| 44|  8324.4|      KATARZYNA|
|  6|      Adam|           Kowalski| 27|  8850.1|           ADAM|
|  7|   Wisława|             Wlotka| 29| 8950.67|        WISŁAWA|
|  8|Mieczysław|Brzęczyszczykiewicz| 32| 6973.38|     MIECZYSŁAW|
|  9| Krzysztof|Brzęczyszczykiewicz| 37|10180.01|      KRZYSZTOF|
| 10| Katarzyna|             Wlotka| 26| 8365.36|      KATARZYNA|
+---+----------+-------------------+---+--------+---------------+
only showing top 10 rows



In [40]:
# filtrowanie numeryczne, ale tu na kolumnie typu str - czy jest poprawne?
df.filter(df["salary"] > 10000).count()

316067

In [41]:
# na ile partycji została nasza ramka danych rozrzucona po "klastrze"?
df.rdd.getNumPartitions()

6

In [42]:
%%time
# mierzymy czas operacji przy domyślnej liczbie partycji
df.filter(df["salary"] > 10000).count()

[Stage 40:================================================>         (5 + 1) / 6]

CPU times: user 15.1 ms, sys: 196 μs, total: 15.3 ms
Wall time: 14.2 s


316067

In [43]:
df = df.repartition(12)

In [44]:
df.rdd.getNumPartitions()

[Stage 43:================================================>         (5 + 1) / 6]

12

In [45]:
%%time
# mierzymy czas operacji przy 12 partycjach dla 20_000_000 rekordów
df.filter(df["salary"] > 10000).count()

[Stage 44:================================================>         (5 + 1) / 6]

CPU times: user 15.9 ms, sys: 614 μs, total: 16.5 ms
Wall time: 15 s


316067

In [46]:
# macierz częstości dla dwóch kolumn - uwaga dla bardzo różnorodnych danych!
df.crosstab("firstname", "age").sort("firstname_age").show()

24/12/16 16:45:44 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
[Stage 65:===================================================>    (11 + 1) / 12]

+-------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|firstname_age|   18|   19|   20|   21|   22|   23|   24|   25|   26|   27|   28|   29|   30|   31|   32|   33|   34|   35|   36|   37|   38|   39|   40|   41|   42|   43|   44|   45|   46|   47|   48|   49|   50|   51|   52|   53|   54|   55|   56|   57|   58|   59|   60|   61|   62|   63|   64|   65|   66|   67|   68|
+-------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|         Adam|39417|39175|39564|3

In [47]:
# funkcja explain może przydać się w przypadku bardziej zaawansowanego debuggingu, optymalizacji i zrozumienia
# kolejności działania niektórych elementów silnika Spark
query = df.filter(df.firstname.contains('ski'))
query.explain(mode='formatted')

== Physical Plan ==
AdaptiveSparkPlan (5)
+- Exchange (4)
   +- Project (3)
      +- Filter (2)
         +- Scan csv  (1)


(1) Scan csv 
Output [5]: [id#1055, firstname#1056, lastname#1057, age#1058, salary#1059]
Batched: false
Location: InMemoryFileIndex [file:/opt/spark/work-dir/Lab6/employee.csv]
PushedFilters: [IsNotNull(firstname), StringContains(firstname,ski)]
ReadSchema: struct<id:int,firstname:string,lastname:string,age:int,salary:double>

(2) Filter
Input [5]: [id#1055, firstname#1056, lastname#1057, age#1058, salary#1059]
Condition : (isnotnull(firstname#1056) AND Contains(firstname#1056, ski))

(3) Project
Output [6]: [id#1055, firstname#1056, lastname#1057, age#1058, salary#1059, upper(firstname#1056) AS firstname_upper#1092]
Input [5]: [id#1055, firstname#1056, lastname#1057, age#1058, salary#1059]

(4) Exchange
Input [6]: [id#1055, firstname#1056, lastname#1057, age#1058, salary#1059, firstname_upper#1092]
Arguments: RoundRobinPartitioning(12), REPARTITION_BY_NUM, [plan

In [48]:
# zapisujemy ramkę do plików parquet
# zwróć uwagę na liczbę utworzonych plików

df.write.mode('overwrite').parquet('./data/parquet/')

In [50]:
sc.stop()